In [7]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns

%matplotlib inline
plt.rcParams["figure.figsize"] = (15, 8)
pd.options.display.float_format = '{:.2f}'.format

from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv('data/vk_users_data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35517 entries, 0 to 35516
Data columns (total 18 columns):
political     5400 non-null float64
country       34114 non-null object
smoking       6264 non-null float64
sex           35517 non-null int64
id            35517 non-null int64
last_name     35517 non-null object
alcohol       6189 non-null float64
religion      6357 non-null object
langs         4624 non-null object
city          32524 non-null object
relation      14826 non-null float64
age           35517 non-null float64
verified      35517 non-null int64
bdate         29173 non-null object
first_name    35517 non-null object
university    14826 non-null float64
life_main     6239 non-null float64
posts         35517 non-null object
dtypes: float64(7), int64(3), object(8)
memory usage: 4.9+ MB


In [8]:
df_train, df_test = train_test_split(df, test_size=0.3)

In [9]:
df_train.head()

,political,country,smoking,sex,id,last_name,alcohol,religion,langs,city,relation,age,verified,bdate,first_name,university,life_main,posts
35147,nan,США,nan,1,8472929,Natasha,nan,NaN,NaN,Chicago,nan,66.00,0,17.7.1950,Natali,nan,nan,нет никаких правил)))\nпредставляю вашему вним...
801,nan,Россия,nan,2,143523733,Калинин,nan,NaN,NaN,Ижевск,nan,110.00,0,24.1,Дмитрий,nan,nan,Пока Дима не против \nЯ на его стене посижу \n...
6000,nan,Россия,nan,1,64085242,Попова,nan,NaN,NaN,Ревда,nan,74.00,0,17.9,Алёна,nan,nan,"Меня всегда удивляло, как неожиданное стечение..."
9052,nan,Украина,nan,2,335527545,Ulanov,nan,NaN,NaN,Харьков,nan,66.00,0,21.10.1953,Anatoliy,nan,nan,Заходите и \nПолучайте \nот меня \nв подарок: ...
6106,nan,Россия,3.00,2,260522138,Варданян,3.00,NaN,"['Русский', 'Հայերեն']",Краснодар,3.00,94.00,0,21.9.1924,Егор,0.00,1.00,♤♡♤\n[id436173529|Эдуард Априкян] \nУдачно слу...


In [10]:
df_test.head()

,political,country,smoking,sex,id,last_name,alcohol,religion,langs,city,relation,age,verified,bdate,first_name,university,life_main,posts
31988,nan,Россия,nan,1,118616259,Соколова,nan,NaN,"['Русский', 'English', 'Українська']",Санкт-Петербург,4.00,18.00,0,17.10.1999,Анастасия,0.00,nan,"Болею сейчас сильно, в шутку назвала стрим ""со..."
27728,nan,Украина,nan,1,105935486,Мищенко,nan,NaN,NaN,Ахтырка,nan,110.00,0,11.7,Анастасия,nan,nan,❗❗❗❗❗❗❗❗❗\nСпасибо всем вам за такой прекрасн...
1195,nan,Украина,nan,2,717510,Май,nan,либеральные,NaN,Харьков,1.00,50.00,0,25.6.1969,Захар,158448.00,nan,"Захар, сколько тебе лет?\nХарьков\nКиев: 22 се..."
12990,nan,Россия,nan,2,38056252,Малородний,nan,NaN,NaN,Москва,nan,86.00,0,27.4.1932,Вячеслав,nan,nan,"Было,стало))\nВот такие дела 😒\nИдёт кто-нить?..."
2796,nan,Россия,nan,2,23162402,Корнелюк,nan,NaN,NaN,Новосибирск,nan,26.00,0,19.3.1992,Алексей,nan,nan,Внимание! Перед вами самый коварный мошенник и...
